# Building Production Ready RAG Pipeline

In this notebook you will learn to build a Production ready RAG Pipeline on `Attention is All You Need` paper. We will use `Sentence Window Index` to build a basic RAG pipeline and iterate over different parameters to make it production ready.

Following are the steps involved:

1. Download Data
2. Load Data
3. Build Evaluation Dataset.
4. Download `RagEvaluatorPack`.
5. Define LLM, Embedding Model.
6. Build RAG with `Sentence Window` approach.
7. Evaluate RAG Pipeline.
8. Create functions to build index, evaluate.
9. Tune different parameters to improve metrics and make it production ready.

## Setup

Install the libraries.

In [43]:
!pip install llama-index pypdf torch sentence-transformers llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.0.1
    Uninstalling sentence-transformers-3.0.1:
      Successfully uninstalled sentence-transformers-3.0.1


## Download `Attention is all you need` paper.

In [44]:
!mkdir './data'
!wget --user-agent="Mozilla" "https://arxiv.org/pdf/1706.03762.pdf" -O "./data/attention_is_all_you_need.pdf"

mkdir: cannot create directory ‘./data’: File exists
--2024-06-08 15:41:28--  https://arxiv.org/pdf/1706.03762.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.195.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/1706.03762 [following]
--2024-06-08 15:41:28--  http://arxiv.org/pdf/1706.03762
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘./data/attention_is_all_you_need.pdf’

./data/attention_is 100%[===================>]   2.11M  --.-KB/s    in 0.08s   

2024-06-08 15:41:28 (26.5 MB/s) - ‘./data/attention_is_all_you_need.pdf’ saved [2215244/2215244]



## Set `OpenAI` keys.

In [45]:
import nest_asyncio

nest_asyncio.apply()

from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY_1')

## Load Data.

We will use first 5 pages and skip paper references in the paper.

In [46]:
from llama_index.core import SimpleDirectoryReader

data = SimpleDirectoryReader('./data/').load_data()

documents = data[:5]

## Generate Evaluation dataset using `RagDatasetGenerator` and `GPT-4`

In [47]:
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

gpt4 = OpenAI(model='gpt-4', temperature=0.1)
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    service_context=service_context_gpt4,
    num_questions_per_chunk=2,
    show_progress=True,
)

eval_dataset = dataset_generator.generate_dataset_from_nodes()

<ipython-input-47-2dd75cd664eb>:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)


Parsing nodes:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 5/5 [00:05<00:00,  1.16s/it]

100%|██████████| 2/2 [00:09<00:00,  4.58s/it]

100%|██████████| 2/2 [00:10<00:00,  5.14s/it]

100%|██████████| 2/2 [00:07<00:00,  3.90s/it]

100%|██████████| 2/2 [00:07<00:00,  3.74s/it]

100%|██████████| 2/2 [00:06<00:00,  3.41s/it]


## Download `RagEvaluatorPack` for evaluation.

In [48]:
from llama_index.core.llama_pack import download_llama_pack

RagEvaluatorPack = download_llama_pack(
  "RagEvaluatorPack", "./rag_evaluator_pack"
)

## Define LLM.

In [49]:
# from llama_index.llms. import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

## Define Embedding Model

In [50]:
embed_model = "local:BAAI/bge-small-en-v1.5"

## Build RAG pipeline with `SentenceWindow`

In [51]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=1,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [52]:
from llama_index.core import ServiceContext

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

<ipython-input-52-b25f321d995e>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [54]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [55]:
from llama_index.core import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [56]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [57]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

In [58]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=2, node_postprocessors=[postproc, rerank]
)

In [59]:
response = query_engine.query('is the paper from google research?')
print(response)

Yes.


## Evaluate RAG pipeline

In [60]:
rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=query_engine
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)



Batch processing of predictions: 100%|██████████| 10/10 [00:17<00:00,  1.74s/it]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:06<00:35,  6.53s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:06<00:35,  6.53s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:14<00:16,  4.78s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:14<00:16,  4.78s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:24<00:07,  4.74s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:24<00:07,  4.74s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:33<00:02,  5.83s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:33<00:02,  5.83s/it]
Batch processing of evaluations: 8it [00:40,  4.97s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:40<00:12,  8.16s/it]


In [61]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,3.750000
mean_relevancy_score,0.900000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.940403


## Create Functions to build RAG pipeline and Evaluation.

This will make the process of iterating easier for evaluation.

In [62]:
def build_index(
    documents,
    llm=OpenAI(model='gpt-3.5-turbo', temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    sentence_index = VectorStoreIndex.from_documents(
        documents, service_context=sentence_context
    )

    return sentence_index


def setup_query_engine(
    sentence_index, similarity_top_k=2, rerank_top_n=2, is_rerank = False
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    if is_rerank:
      rerank = SentenceTransformerRerank(
          top_n=rerank_top_n, model="BAAI/bge-reranker-base"
      )
      query_engine = sentence_index.as_query_engine(
          similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
      )
    else:
      query_engine = sentence_index.as_query_engine(
          similarity_top_k=similarity_top_k, node_postprocessors=[postproc]
      )
    return query_engine

async def evaluate(query_engine):
  rag_evaluator_pack = RagEvaluatorPack(
      rag_dataset=eval_dataset,
      query_engine=query_engine
  )

  benchmark_df = await rag_evaluator_pack.arun(
      batch_size=10,  # batches the number of openai api calls to make
      sleep_time_in_seconds=1,  # seconds to sleep before making an api call
  )

  return benchmark_df

def build_index_query_engine(window_size,
                             similarity_top_k,
                             rerank_similarity_top_k,
                             rerank_top_k):
  sentence_index = build_index(
      [document],
      sentence_window_size=window_size,
  )

  query_engine = setup_query_engine(sentence_index,
                                      similarity_top_k=similarity_top_k)

  query_engine_rerank = setup_query_engine(sentence_index,
                                            similarity_top_k=rerank_similarity_top_k,
                                            rerank_top_n=rerank_top_k,
                                            is_rerank=True)
  return sentence_index, query_engine, query_engine_rerank

In [63]:
index, query_engine, query_engine_rerank = build_index_query_engine(1, 2, 4, 2)

<ipython-input-62-cc9a37eba68d>:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [64]:
base_benchmark = await evaluate(query_engine)


Batch processing of predictions: 100%|██████████| 10/10 [00:04<00:00,  2.13it/s]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:05<00:30,  5.49s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:05<00:30,  5.49s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:12<00:14,  4.00s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:12<00:14,  4.00s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:20<00:06,  4.14s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:21<00:06,  4.14s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:28<00:02,  4.92s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:28<00:02,  4.92s/it]
Batch processing of evaluations: 8it [00:36,  4.70s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:36<00:11,  7.38s/it]


In [65]:
base_benchmark_rerank = await evaluate(query_engine_rerank)


Batch processing of predictions: 100%|██████████| 10/10 [00:30<00:00,  3.04s/it]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:07<00:43,  7.90s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:07<00:43,  7.90s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:15<00:17,  4.86s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:15<00:17,  4.86s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:21<00:06,  4.02s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:21<00:06,  4.02s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:27<00:02,  4.52s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:27<00:02,  4.52s/it]
Batch processing of evaluations: 8it [00:34,  4.07s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:34<00:10,  6.96s/it]


In [66]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.150000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.930138


In [67]:
base_benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,3.650000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.935733


From the metrics we can observe that `correctness` metric is lower (maximum 5) and including `reranker` improved metrics though it decreased the `correctness` metric.

Interesting to see there are no hallucinations as `faithfulness` metric is 1.0

## Tune parameters to make it production ready.

Let's aim to get `correctness` score of `4.5` and `relevancy` score of more than `0.9`.

### Experiment 1:

Let's increase window size and see if we can improve correctness as it gives more surrounding context.

In [68]:
index, query_engine, query_engine_rerank = build_index_query_engine(3, 2, 4, 2)

<ipython-input-62-cc9a37eba68d>:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [69]:
benchmark = await evaluate(query_engine)


Batch processing of predictions: 100%|██████████| 10/10 [00:04<00:00,  2.03it/s]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:05<00:28,  5.22s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:05<00:28,  5.22s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:12<00:13,  3.91s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:12<00:13,  3.91s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:19<00:05,  3.83s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:19<00:05,  3.83s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:27<00:02,  4.80s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:27<00:02,  4.80s/it]
Batch processing of evaluations: 8it [00:34,  4.35s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:34<00:10,  6.92s/it]


In [70]:
benchmark_rerank = await evaluate(query_engine_rerank)


Batch processing of predictions: 100%|██████████| 10/10 [00:58<00:00,  5.89s/it]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:05<00:30,  5.54s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:05<00:30,  5.54s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:12<00:13,  3.99s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:12<00:13,  3.99s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:18<00:05,  3.52s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:18<00:05,  3.52s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:26<00:02,  4.81s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:26<00:02,  4.81s/it]
Batch processing of evaluations: 8it [00:34,  4.28s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:34<00:10,  6.82s/it]


In [71]:
benchmark

rag,base_rag
metrics,
mean_correctness_score,4.200000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.946689


In [72]:
benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,4.100000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.954488


The metrics did improve.

### Experiment 2:

Let'r increase `similarity_top_k` and `reranker_top_n` values and see if getting more relevant contexts improves the result.

In [73]:
index, query_engine, query_engine_rerank = build_index_query_engine(3, 4, 8, 4)

<ipython-input-62-cc9a37eba68d>:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [74]:
benchmark = await evaluate(query_engine)


Batch processing of predictions: 100%|██████████| 10/10 [00:04<00:00,  2.14it/s]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:05<00:29,  5.35s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:05<00:29,  5.35s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:13<00:14,  4.26s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:13<00:14,  4.26s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:19<00:05,  3.82s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:19<00:05,  3.82s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:26<00:02,  4.52s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:26<00:02,  4.52s/it]
Batch processing of evaluations: 8it [00:33,  4.15s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:33<00:10,  6.73s/it]


In [75]:
benchmark_rerank = await evaluate(query_engine_rerank)


Batch processing of predictions: 100%|██████████| 10/10 [02:06<00:00, 12.66s/it]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:06<00:35,  6.40s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:06<00:35,  6.40s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:13<00:14,  4.22s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:13<00:14,  4.22s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:21<00:06,  4.09s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:21<00:06,  4.09s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:28<00:02,  4.90s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:28<00:02,  4.90s/it]
Batch processing of evaluations: 8it [00:35,  4.21s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:35<00:10,  7.00s/it]


In [76]:
benchmark

rag,base_rag
metrics,
mean_correctness_score,4.300000
mean_relevancy_score,0.900000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.948439


In [77]:
benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,4.200000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.953095


We are close to our goal. We reached a `correctness` score of `4.275` and `relevancy` score of 0.9. The `context similarity score` also improved

### Experiment 3:

Let's now increase window size and see if it improves the metrics.

In [78]:
index, query_engine, query_engine_rerank = build_index_query_engine(5, 4, 8, 4)

<ipython-input-62-cc9a37eba68d>:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [79]:
benchmark = await evaluate(query_engine)


Batch processing of predictions: 100%|██████████| 10/10 [00:05<00:00,  1.79it/s]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:06<00:36,  6.62s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:06<00:36,  6.62s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:13<00:14,  4.12s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:13<00:14,  4.12s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:20<00:05,  3.78s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:20<00:05,  3.78s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:27<00:02,  4.77s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:27<00:02,  4.77s/it]
Batch processing of evaluations: 8it [00:33,  4.11s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:33<00:10,  6.80s/it]


In [80]:
benchmark_rerank = await evaluate(query_engine_rerank)


Batch processing of predictions: 100%|██████████| 10/10 [03:27<00:00, 20.76s/it]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:04<00:25,  4.71s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:04<00:25,  4.71s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:11<00:12,  3.64s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:11<00:12,  3.64s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:20<00:06,  4.02s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:20<00:06,  4.02s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:26<00:02,  4.61s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:26<00:02,  4.61s/it]
Batch processing of evaluations: 8it [00:33,  4.17s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:33<00:10,  6.68s/it]


In [81]:
benchmark

rag,base_rag
metrics,
mean_correctness_score,4.150000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.953137


In [82]:
benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,4.15000
mean_relevancy_score,1.00000
mean_faithfulness_score,1.00000
mean_context_similarity_score,0.95222


We have reached a `relevency` score of `1.0`.

### Experiment 4:

Let'r increase `similarity_top_k` and `reranker_top_n` values and see if getting more relevant contexts improves the result.

In [83]:
index, query_engine, query_engine_rerank = build_index_query_engine(5, 6, 12, 6)

<ipython-input-62-cc9a37eba68d>:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [84]:
benchmark_rerank = await evaluate(query_engine_rerank)


Batch processing of predictions: 100%|██████████| 10/10 [05:00<00:00, 30.06s/it]

Batch processing of evaluations:   0%|          | 0/6.5 [00:00<?, ?it/s]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:04<00:27,  4.93s/it]
Batch processing of evaluations:  15%|█▌        | 1/6.5 [00:04<00:27,  4.93s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:10<00:12,  3.49s/it]
Batch processing of evaluations:  46%|████▌     | 3/6.5 [00:10<00:12,  3.49s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:19<00:05,  3.82s/it]
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:19<00:05,  3.82s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:26<00:02,  4.78s/it]
Batch processing of evaluations:  92%|█████████▏| 6/6.5 [00:26<00:02,  4.78s/it]
Batch processing of evaluations: 8it [00:32,  3.98s/it]                         
Batch processing of evaluations:  77%|███████▋  | 5/6.5 [00:32<00:09,  6.49s/it]


In [85]:
benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,4.400000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.953447


We have reached our goal of `correctness` score of `4.5` and `relevancy` score of `1.0` (>0.9).

## Observation:

In this project, we looked into building RAG Pipeline, evaluation dataset and tuning different parameters to make it production ready. It should be observed that `reranker` improved metrics in most of the experiments.

Please do remember that we have various other metrics like `chunk_size`, `chunk_overlap`, `embedding model`, `LLM` to experiment.